In [1]:
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

import torch
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

from data_utility import *
from data_utils import *
from loss import *
from train import *
from deeplab_model.deeplab import *
from dense_vnet.DenseVNet import DenseVNet
from sync_batchnorm import convert_model
import datetime

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
USE_GPU = True
NUM_WORKERS = 12
BATCH_SIZE = 2 

dtype = torch.float32 
# define dtype, float is space efficient than double

if USE_GPU and torch.cuda.is_available():
    
    device = torch.device("cuda:0")
    
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.enabled = True
    # magic flag that accelerate
    
    print('using GPU for training')
else:
    device = torch.device('cpu')
    print('using CPU for training')

using GPU for training


In [3]:
train_dataset = get_full_resolution_dataset(data_type = 'nii_train', 
                transform=transforms.Compose([
                random_affine(90, 15),
                random_filp(0.5)]))
# do data augumentation on train dataset

validation_dataset = get_full_resolution_dataset(data_type = 'nii_test', 
                transform=None)
# no data augumentation on validation dataset

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,
                    num_workers=NUM_WORKERS)
validation_loader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=True,
                    num_workers=NUM_WORKERS) # drop_last
# loaders come with auto batch division and multi-thread acceleration

In [4]:
from bv_refinement_network.RefinementModel import RefinementModel, RefinementModel_NoDown

if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
    
refine_model = RefinementModel(num_classes=1)
refine_model = nn.DataParallel(refine_model)
refine_model = convert_model(refine_model)
refine_model = refine_model.to(device, dtype)

optimizer = optim.Adam(refine_model.parameters(), lr=1e-2)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=25)

deeplab = DeepLab(output_stride=16)
deeplab = nn.DataParallel(deeplab)
deeplab = convert_model(deeplab)

checkpoint = torch.load('../deeplab_dilated_save/2019-08-10 09:28:43.844872 epoch: 1160.pth') # best one

deeplab.load_state_dict(checkpoint['state_dict_1'])
deeplab = deeplab.to(device, dtype)

epoch = 0

Let's use 2 GPUs!


In [ ]:
'''
test_dictionary = train_dataset[33]

image_1 = test_dictionary['image1_data'].view(1, 1, 256, 256, 256)
label_1 = test_dictionary['image1_label'].view(1, 3, 256, 256, 256)
bv_label = label_1.narrow(1,2,1).to(device, dtype)
if get_dimensions(bv_label) == 4:
    bv_label.unsqueeze_(0)

image_1 = image_1.to(device=device, dtype=dtype) 
label_1 = label_1.to(device=device, dtype=dtype)
'''

"\ntest_dictionary = train_dataset[33]\n\nimage_1 = test_dictionary['image1_data'].view(1, 1, 256, 256, 256)\nlabel_1 = test_dictionary['image1_label'].view(1, 3, 256, 256, 256)\nbv_label = label_1.narrow(1,2,1).to(device, dtype)\nif get_dimensions(bv_label) == 4:\n    bv_label.unsqueeze_(0)\n\nimage_1 = image_1.to(device=device, dtype=dtype) \nlabel_1 = label_1.to(device=device, dtype=dtype)\n"

In [ ]:
def get_bboxes(image, label, output, batchsize):
    image_final = torch.zeros((batchsize, 1, 128, 128, 128)).to(device, dtype)
    label_final = torch.zeros((batchsize, 1, 128, 128, 128)).to(device, dtype)
    output_final = torch.zeros((batchsize, 1, 128, 128, 128)).to(device, dtype)
    for b in range(batchsize):
        out = output[b]
        x, y, z = find_bv_centroid(binarize_output(out))
        #x,y,z = loadbvcenter(binarize_output(out))
        x, y, z = np.clip([x, y, z], a_min=64, a_max=192)
        x1 = max(x-64, 0)
        x2 = min(x+64, 256)
        y1 = max(y-64, 0)
        y2 = min(y+64, 256)
        z1 = max(z-64, 0)
        z2 = min(z+64, 256)
        image_final[b] = image[b, :, x1:x2, y1:y2, z1:z2]
        label_final[b] = label[b, :, x1:x2, y1:y2, z1:z2]
        output_final[b] = output[b, :, x1:x2, y1:y2, z1:z2]
    return image_final, label_final, output_final

In [ ]:
epochs = 5000

record = open('train_bv_refine_5.txt','a+')

logger = {'train':[], 'validation_1': []}

min_val = 1

for e in tqdm(range(epoch + 1, epochs)):
# iter over epoches
    epoch_loss = 0
        
    for t, batch in enumerate(train_loader):
    # iter over the train mini batches
        refine_model.train()
        deeplab.eval()
        # Set the model flag to train
        # 1. enable dropout
        # 2. batchnorm behave differently in train and test
        #print(batch['image1_data'])
        image_1 = batch['image1_data'].to(device=device, dtype=dtype)
        image_1 = image_1.view(BATCH_SIZE,1,256,256,256)

        label_1 = batch['image1_label'].to(device=device, dtype=dtype)
        label_1 = label_1.view(BATCH_SIZE,3,256,256,256)

        bv_label = label_1[:, 2, :, :, :]
        bv_label = bv_label.view(BATCH_SIZE,1,256,256,256)

        #original_res = [a[minibatch].item() for a in batch['original_resolution']]

        # Get coarse output from deeplab model from 256 resolution input
        out_coarse = deeplab(image_1)
        out_coarse = out_coarse.view(BATCH_SIZE,3,256,256,256)

        bv_coarse = out_coarse[:, 2, :, :, :]
        bv_coarse = bv_coarse.view(BATCH_SIZE,1,256,256,256)

        bbox_image, bbox_label, bbox_bv = get_bboxes(image_1, bv_label, bv_coarse, BATCH_SIZE)
        #bbox_image, bbox_label, bbox_bv = get_bboxes(image_1, bv_label, bv_coarse, BATCH_SIZE)
        
        bbox_concat = torch.cat([bbox_image, bbox_bv], dim=1)
        
        #show_image_slice(label_1)
        #show_image_slice(bv_label)
        #show_image_slice(bbox_image)
        #show_image_slice(bbox_label)
        #show_image_slice(bbox_bv)

        #print("bbox_concat", bbox_concat.shape)

        del out_coarse
        del image_1
        del bv_coarse
        del label_1
        del bv_label
        del bbox_image
        del bbox_bv

        torch.cuda.empty_cache()

        refine_out = refine_model(bbox_concat)
        # do the inference

        #print(refine_out.shape)
        #print(bbox_bv_label.shape)

        loss = dice_loss(refine_out, bbox_label)
        
        print(loss)
        epoch_loss += loss.item()
        # record minibatch loss to epoch loss
        
        optimizer.zero_grad()
        # set the model parameter gradient to zero
        
        loss.backward()
        # calculate the gradient wrt loss
        optimizer.step()
        # take a gradient descent step
        torch.cuda.empty_cache()
        
    outstr = 'Epoch {0} finished ! Training Loss: {1:.4f}'.format(e, epoch_loss/(t+1)) + '\n'
    
    logger['train'].append(epoch_loss/(t+1))
    
    print(outstr)
    record.write(outstr)
    record.flush()

    if e%2 == 0:
    # do validation every 5 epoches
        deeplab.eval()
        refine_model.eval()
        # set model flag to eval
        # 1. disable dropout
        # 2. batchnorm behave differs

        with torch.no_grad():
        # stop taking gradient
        
            #valloss_4 = 0
            #valloss_2 = 0
            valloss_1 = 0
            
            for v, vbatch in enumerate(validation_loader):
                image_1 = vbatch['image1_data'].to(device=device, dtype=dtype)
                image_1 = image_1.view(BATCH_SIZE,1,256,256,256)

                label_1 = vbatch['image1_label'].to(device=device, dtype=dtype)
                label_1 = label_1.view(BATCH_SIZE,3,256,256,256)

                bv_label = label_1[:, 2, :, :, :]
                bv_label = bv_label.view(BATCH_SIZE,1,256,256,256)

                #original_res = [a[minibatch].item() for a in batch['original_resolution']]

                # Get coarse output from deeplab model from 256 resolution input
                out_coarse = deeplab(image_1)
                out_coarse = out_coarse.view(BATCH_SIZE,3,256,256,256)

                bv_coarse = out_coarse[:, 2, :, :, :]
                bv_coarse = bv_coarse.view(BATCH_SIZE,1,256,256,256)

                bbox_image, bbox_label, bbox_bv = get_bboxes(image_1, bv_label, bv_coarse, BATCH_SIZE)

                bbox_concat = torch.cat([bbox_image, bbox_bv], dim=1)

                #show_image_slice(label)
                #show_image_slice(bbox_image)
                #show_image_slice(bbox_label)
                #show_image_slice(bbox_bv)

                #print("bbox_concat", bbox_concat.shape)
                #seg_image_concat = torch.cat([bv_coarse, image_1], dim=1)

                del out_coarse
                del image_1
                del bv_coarse
                del label_1
                del bv_label
                del bbox_image
                del bbox_bv

                torch.cuda.empty_cache()

                refine_out = refine_model(bbox_concat)
                        
                loss = dice_loss(refine_out, bbox_label)
                
                print(loss)
            
                # calculate loss
                valloss_1 += loss.item()
                
                torch.cuda.empty_cache()
                
            
            avg_val_loss = (valloss_1 / (v+1))
            outstr = '------- 1st valloss={0:.4f}'\
                .format(avg_val_loss) + '\n'
            
            logger['validation_1'].append(avg_val_loss)
            #scheduler.step(avg_val_loss)
            
            if avg_val_loss < min_val:
                save_1('refine_bv5_save', refine_model, optimizer, logger, e, scheduler)
            elif e % 10 == 0:
                save_1('refine_bv5_save', refine_model, optimizer, logger, e, scheduler)
            
            torch.cuda.empty_cache()
            
            print(outstr)
            record.write(outstr)
            record.flush()
    


record.close()

  0%|          | 0/4999 [00:00<?, ?it/s]

tensor(0.9966, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9981, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9234, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9878, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.8701, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.5934, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.6249, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.7340, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9960, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9872, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9479, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9996, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9802, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(1.0000, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9997, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9999, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(1.0000, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.6811, device='cuda:0',

  0%|          | 1/4999 [12:49<1068:37:03, 769.71s/it]

Epoch 1 finished ! Training Loss: 0.7244

tensor(0.5967, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.5123, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.1781, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9999, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.5242, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.6567, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.6049, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.3090, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.3792, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.3967, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.5835, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.6485, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9999, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9911, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.6993, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.7677, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.2983, device='cuda:0', grad_fn=<RsubB

  0%|          | 2/4999 [25:29<1064:07:52, 766.63s/it]

...

Epoch 15 finished ! Training Loss: 0.4950

tensor(0.3074, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.6207, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.2267, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9896, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.2227, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.3027, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.5562, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.4826, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.2611, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.5648, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9978, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.2424, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.7194, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.5954, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.1573, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.2692, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.5779, device='cuda:0', grad_fn=<Rsub

  0%|          | 16/4999 [3:16:14<1021:45:07, 738.17s/it]

Checkpoint 16 saved !
------- 1st valloss=0.2805

tensor(0.2619, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.4333, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.3462, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.2999, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.5901, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.5966, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.5544, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.7772, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.1818, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.6128, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.3032, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.7322, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.5885, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.3265, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9975, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.2226, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.3142, device='cuda:0', grad_f

  0%|          | 17/4999 [3:27:52<1005:03:19, 726.25s/it]

Epoch 17 finished ! Training Loss: 0.5115

tensor(0.6856, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(1., device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.2068, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.6858, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.2749, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.6309, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.2399, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.6016, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.4188, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.6551, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.7990, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.2130, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.6187, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.3607, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.5751, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(1., device='cuda:0', grad_fn=<RsubBackward1>)
tensor(1., device='cuda:0', grad_fn=<RsubBackward1>)


In [ ]:
deeplab.eval()

with torch.no_grad():
    
    bgloss = 0
    bdloss = 0
    bvloss = 0
    
    for v, vbatch in tqdm(enumerate(validation_loader)):
        # move data to device, convert dtype to desirable dtype
        image_1 = vbatch['image1_data'].to(device=device, dtype=dtype)
        label_1 = vbatch['image1_label'].to(device=device, dtype=dtype)

        output = deeplab(image_1)
        # do the inference
        output_numpy = output.cpu().numpy()
        
        
        #out_1 = torch.round(output)
        out_1 = torch.from_numpy((output_numpy == output_numpy.max(axis=1)[:, None]).astype(int)).to(device=device, dtype=dtype)
        loss_1 = dice_loss_3(out_1, label_1)

        bg, bd, bv = dice_loss_3_debug(out_1, label_1)
        # calculate loss
        print(bg.item(), bd.item(), bv.item(), loss_1.item())
        bgloss += bg.item()
        bdloss += bd.item()
        bvloss += bv.item()
        
        if bv.item() >= 0.2 or bd.item() >= 0.1:
            show_image_slice(image_1)
            show_image_slice(label_1)
            show_image_slice(output)

    outstr = '------- background loss = {0:.4f}, body loss = {1:.4f}, bv loss = {2:.4f}'\
        .format(bgloss/(v+1), bdloss/(v+1), bvloss/(v+1)) + '\n'
    print(outstr)